In [2]:
import csv
import json

# Open the CSV file
with open('pervalence data 2021.csv', 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    
    # Convert each row to a dictionary
    data = []
    for row in csvreader:
        data.append(row)

# Write the data to a JSON file
with open('prevalence_data.json', 'w') as jsonfile:
    json.dump(data, jsonfile, indent=4)

In [7]:
import csv
import json

def csv_to_nested_json(csv_file):
  """Converts CSV prevalence data to nested JSON format.

  Args:
    csv_file: Path to the CSV file.

  Returns:
    A JSON string containing the nested prevalence data.
  """
  diseases = {}

  with open(csv_file, 'r', encoding='utf-8-sig') as file:
    reader = csv.DictReader(file)
    for row in reader:
      disease_id = row['disease id']
      disease_name = row['disease']
      sex = row['sex'].lower()
      age_group = row['age group']
      percent = float(row['percent'])

      if disease_id not in diseases:
        diseases[disease_id] = {
            'id': disease_id,
            'name': disease_name,
            'prevalence': {'male': {}, 'female': {}}
        }

      diseases[disease_id]['prevalence'][sex][age_group] = percent

  return json.dumps({'diseases': list(diseases.values())}, indent=2)

# Example usage:
json_data = csv_to_nested_json('Prevalence data 2021 .csv')
# Write the data to a JSON file
with open('prevalence_data.json', 'w') as jsonfile:
    json.dump(data, json_data, indent=4)

AttributeError: 'str' object has no attribute 'write'

In [17]:
import pandas as pd

# Load the Google Sheets data into a Pandas DataFrame
df = pd.read_csv('/Users/sunny/Documents/GitHub/EM-bot-refresh/Prevalence of disease.csv')

# Group the data by 'Disease ID' and 'Disease' columns
grouped = df.groupby(['disease id', 'disease'])

# Create the nested JSON structure
diseases = []
for (disease_id, disease_name), group in grouped:
    
    # Get the associated names for the disease
    associated_names = list(group['associated names'].unique())

    # Create prevalence dictionary with age groups nested under male/female
    prevalence = {}
    for sex in ['Male', 'Female']:
        prevalence[sex.lower()] = group.loc[group['sex'] == sex, ['age group', 'percent']].to_dict(orient='records')

    # Append disease dictionary to the list
    diseases.append({
        'id': int(disease_id),  # Convert disease_id to int
        'name': disease_name,
        'associated_names': associated_names,
        'prevalence': prevalence
    })

# Create the final JSON object
json_data = {'diseases': diseases}

with open('diseases.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)

In [25]:
import pandas as pd

# Load the Google Sheets data into a Pandas DataFrame
df = pd.read_csv('/Users/sunny/Documents/GitHub/EM-bot-refresh/Prevalence of disease.csv')

# Group the data by 'Disease ID' and 'Disease' columns
grouped = df.groupby(['disease id', 'disease'])

# Create the nested JSON structure
diseases = []
for (disease_id, disease_name), group in grouped:
    
    # Get the associated names for the disease
    associated_names = list(group['associated names'].unique())

    # Create prevalence dictionary with age groups nested under male/female
    prevalence = {}
    for sex in ['Male', 'Female']:
        age_groups = []
        for _, row in group.loc[group['sex'] == sex].iterrows():
            age_range = row['age group'].split('-')
            min_age_value = age_range[0].strip()
            max_age_value = age_range[1].strip() if len(age_range) > 1 else None
            
            # Extract age unit from the minimum age value
            age_unit = None
            if 'years' in min_age_value:
                age_unit = 'years'
            elif 'months' in min_age_value:
                age_unit = 'months'
            elif 'days' in min_age_value:
                age_unit = 'days'
            else:
                age_unit = min_age_value.split()[1] if len(min_age_value.split()) > 1 else None
            
            # Extract numeric values for min_age and max_age
            min_age = int(min_age_value.replace('years', '').replace('months', '').replace('days', '').strip().rstrip('+'))
            if max_age_value:
                max_age = int(max_age_value.replace('years', '').replace('months', '').replace('days', '').strip().rstrip('+')) if '+' not in max_age_value else None
            else:
                max_age = None
                
            if age_unit is None:
                continue  # Skip this age group if the unit is not recognized
                
            age_groups.append({
                'min_age': min_age,
                'max_age': max_age,
                'age_unit': age_unit,
                'prevalence': row['percent']
            })
        prevalence[sex.lower()] = age_groups

    # Append disease dictionary to the list
    diseases.append({
        'id': int(disease_id),
        'name': disease_name,
        'associated_names': associated_names,
        'prevalence': prevalence
    })

# Create the final JSON object
json_data = {'diseases': diseases}

with open('diseases2.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)
